## yi = gamma*xi+beta   -> BN γ,β(xi)

In [1]:
from mxnet import nd


/home/wk/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/wk/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
def pure_batch_norm(X, gamma,beta,esp = 1e-5):
    assert len(X.shape) in (2,4)# 维度在（2，4）里
    #fc: batch_size * feature
    if len(X.shape) == 2:
        mean = X.mean(axis = 0)
        variance = ((X - mean)**2).mean(axis = 0)
    #conv-2d: batch_size*channel*h*w
    else:
        mean = X.mean(axis=(0,2,3),keepdims=True)
        variance = ((X-mean)**2).mean(axis = (0,2,3),keepdims = True)
    # normilazition
    X_hat = (X - mean) / nd.sqrt(variance + esp)
    return gamma.reshape(mean.shape) * X_hat + beta.reshape(mean.shape)

In [3]:
a = nd.arange(6).reshape((3,2))
a


[[0. 1.]
 [2. 3.]
 [4. 5.]]
<NDArray 3x2 @cpu(0)>

In [4]:
pure_batch_norm(a,gamma=nd.array([1,1]),beta=nd.array([0,0]))


[[-1.2247427 -1.2247427]
 [ 0.         0.       ]
 [ 1.2247427  1.2247427]]
<NDArray 3x2 @cpu(0)>

In [5]:
b = nd.arange(18).reshape((1,2,3,3))
b



[[[[ 0.  1.  2.]
   [ 3.  4.  5.]
   [ 6.  7.  8.]]

  [[ 9. 10. 11.]
   [12. 13. 14.]
   [15. 16. 17.]]]]
<NDArray 1x2x3x3 @cpu(0)>

In [6]:
bb = b.mean(axis=(0,2,3),keepdims = True)
bb


[[[[ 4.]]

  [[13.]]]]
<NDArray 1x2x1x1 @cpu(0)>

In [7]:
(b - ((b-bb)**2).mean(axis=(0,2,3),keepdims=True))



[[[[-6.6666665 -5.6666665 -4.6666665]
   [-3.6666665 -2.6666665 -1.6666665]
   [-0.6666665  0.3333335  1.3333335]]

  [[ 2.3333335  3.3333335  4.3333335]
   [ 5.3333335  6.3333335  7.3333335]
   [ 8.333334   9.333334  10.333334 ]]]]
<NDArray 1x2x3x3 @cpu(0)>

In [51]:
def batch_norm(X, gamma, beta, is_training, moving_mean, moving_variance,
               eps = 1e-5, moving_momentum = 0.9):
    assert len(X.shape) in (2, 4)
    # 全连接: batch_size x feature
    if len(X.shape) == 2:
        # 每个输入维度在样本上的平均和方差
        mean = X.mean(axis=0)
        variance = ((X - mean)**2).mean(axis=0)
    # 2D卷积: batch_size x channel x height x width
    else:
        # 对每个通道算均值和方差，需要保持4D形状使得可以正确的广播
        mean = X.mean(axis=(0,2,3), keepdims=True)
        variance = ((X - mean)**2).mean(axis=(0,2,3), keepdims=True)
        # 变形使得可以正确的广播
        moving_mean = moving_mean.reshape(mean.shape)
        moving_variance = moving_variance.reshape(mean.shape)

    # 均一化
    if is_training:
        X_hat = (X - mean) / nd.sqrt(variance + eps)
        #!!! 更新全局的均值和方差
        moving_mean[:] = moving_momentum * moving_mean + (
            1.0 - moving_momentum) * mean
        moving_variance[:] = moving_momentum * moving_variance + (
            1.0 - moving_momentum) * variance
    else:
        #!!! 测试阶段使用全局的均值和方差
        X_hat = (X - moving_mean) / nd.sqrt(moving_variance + eps)

    # 拉升和偏移
    return gamma.reshape(mean.shape) * X_hat + beta.reshape(mean.shape)

In [52]:
import utils1
import sys
sys.path.append('.')
ctx = utils1.try_gpu()
ctx

gpu(0)

In [53]:
weight_scale = .01

# 输出通道 = 20, 卷积核 = (5,5)
c1 = 20
W1 = nd.random.normal(shape=(c1,1,5,5), scale=weight_scale, ctx=ctx)
b1 = nd.zeros(c1, ctx)

# 第1层批量归一化
gamma1 = nd.random.normal(shape=c1, scale=weight_scale, ctx=ctx)
beta1 = nd.random.normal(shape=c1, scale=weight_scale, ctx=ctx)
moving_mean1 = nd.zeros(c1, ctx=ctx)
moving_variance1 = nd.zeros(c1, ctx=ctx)

# 输出通道 = 50, 卷积核 = (3,3)
c2 = 50
W2 = nd.random_normal(shape=(c2,c1,3,3), scale=weight_scale, ctx=ctx)
b2 = nd.zeros(c2, ctx=ctx)

# 第2层批量归一化
gamma2 = nd.random.normal(shape=c2, scale=weight_scale, ctx=ctx)
beta2 = nd.random.normal(shape=c2, scale=weight_scale, ctx=ctx)
moving_mean2 = nd.zeros(c2, ctx=ctx)
moving_variance2 = nd.zeros(c2, ctx=ctx)

# 输出维度 = 128
o3 = 128
W3 = nd.random.normal(shape=(1250, o3), scale=weight_scale, ctx=ctx)
b3 = nd.zeros(o3, ctx=ctx)

# 输出维度 = 10
W4 = nd.random_normal(shape=(W3.shape[1], 10), scale=weight_scale, ctx=ctx)
b4 = nd.zeros(W4.shape[1], ctx=ctx)

# 注意这里moving_*是不需要更新的
params = [W1, b1, gamma1, beta1,
          W2, b2, gamma2, beta2,
          W3, b3, W4, b4]

for param in params:
    param.attach_grad()

In [69]:
def net(X, is_training=False, verbose=False):
    X = X.as_in_context(W1.context)
    # 第一层卷积
    h1_conv = nd.Convolution(
        data=X, weight=W1, bias=b1, kernel=W1.shape[2:], num_filter=c1)
    ### 添加了批量归一化层
    h1_bn = batch_norm(h1_conv, gamma1, beta1, is_training,
                       moving_mean1, moving_variance1)
    h1_activation = nd.relu(h1_bn)
    h1 = nd.Pooling(
        data=h1_activation, pool_type="max", kernel=(2,2), stride=(2,2))
    # 第二层卷积
    h2_conv = nd.Convolution(
        data=h1, weight=W2, bias=b2, kernel=W2.shape[2:], num_filter=c2)
    ### 添加了批量归一化层
    h2_bn = batch_norm(h2_conv, gamma2, beta2, is_training,
                       moving_mean2, moving_variance2)
    h2_activation = nd.relu(h2_bn)
    h2_1 = nd.Pooling(data=h2_activation, pool_type="max", kernel=(2,2), stride=(2,2))
    print(h2_1.shape)
    h2 = nd.flatten(h2_1)
    # 第一层全连接
    h3_linear = nd.dot(h2, W3) + b3
    h3 = nd.relu(h3_linear)
    # 第二层全连接
    h4_linear = nd.dot(h3, W4) + b4
    if verbose:
        print('X shape:',X.shape)
        print('1st conv block:', h1.shape)
        print('2nd conv block:', h2.shape)
        print('1st dense:', h3.shape)
        print('2nd dense:', h4_linear.shape)
        print('output:', h4_linear)
    return h4_linear

In [70]:

from mxnet import autograd
from mxnet import gluon

batch_size = 256
train_data, test_data = utils1.load_data_fashion_mnist(batch_size)

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

learning_rate = 0.2

for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data, is_training=True,verbose=True)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        utils1.SGD(params, learning_rate/batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += utils1.accuracy(output, label)

    test_acc = utils1.evaluate_accuracy(test_data, net, ctx)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss / len(train_data), train_acc / len(train_data), test_acc))

/home/wk/anaconda3/lib/python3.6/site-packages/mxnet/gluon/data/vision.py:118: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  label = np.fromstring(fin.read(), dtype=np.uint8).astype(np.int32)
/home/wk/anaconda3/lib/python3.6/site-packages/mxnet/gluon/data/vision.py:122: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(fin.read(), dtype=np.uint8)


(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[ 3.3226438   0.55648565  1.1047484  ... -7.7686048  -0.18948282
  -4.118606  ]
 [-3.0754209  -6.157082   -5.004125   ... 13.681784    1.3849301
  15.09064   ]
 [ 1.8267119  -2.0942879   4.3403563  ... -6.2783604   0.8430075
  -2.912457  ]
 ...
 [-1.4031737  -3.3521314   7.2777963  ... -7.6908836  -3.034771
  -2.7013173 ]
 [ 2.895566   -5.484414    0.17000027 ... -1.3809717  16.215284
   0.84362197]
 [ 2.445634    7.8176727   4.549662   ... -7.4029183  -1.0358758
  -2.23016   ]]
<NDArray 256x10 @gpu(0)>
(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[ 6.7508283  -2.7462177   1.0661666  ... -4.8506684  -0.83463615
  -3.6621792 ]
 [-0.961875   -2.0339618   5.7513356  ... -6.615541   -1.6590009
  -4.0898676 ]
 [ 1.3304381   1.

(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[-1.5656016  -3.1382477   6.987457   ... -6.325817   -3.4083529
  -1.5068843 ]
 [-3.0309772  -4.1127806  -4.431954   ... 16.68346     0.28457934
   3.702018  ]
 [ 2.4785545  -4.9463353  -0.47437525 ...  0.22222373 11.6147995
  -0.34208864]
 ...
 [ 5.553335   -2.808829   -0.92982864 ... -4.311781    0.9440865
  -4.2028584 ]
 [-1.8734133  -5.558068   -4.187831   ... 13.870727    1.4370037
   4.817663  ]
 [ 2.426863   21.575853   -3.7584195  ... -6.9874597  -1.0112681
  -8.216141  ]]
<NDArray 256x10 @gpu(0)>
(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[-2.5161364  -3.1894038  -3.1714947  ... 11.523596    0.37495458
   2.495547  ]
 [ 1.4522767  -5.4428     -6.326669   ...  5.8784294   1.2480313
  -3.275504  ]
 [-1.3204993  -

(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[-1.3147055  -6.178179   -7.2809777  ...  9.809149    1.5017995
  15.602252  ]
 [ 0.6209419   6.5233226  -4.692558   ... -3.0667074  -2.7860901
  -4.479366  ]
 [ 2.9277372  -0.99804056 -8.656464   ... -3.6539984  -3.2202234
   5.1532497 ]
 ...
 [-1.5651816  -5.5499487  -4.560722   ...  7.865349    1.924424
  14.174348  ]
 [ 2.6991513  14.204342   -1.3055624  ... -6.1301746  -2.1142535
  -2.83064   ]
 [-2.1913867  -5.5192065  -3.4710853  ...  8.139099    0.55834806
  16.025702  ]]
<NDArray 256x10 @gpu(0)>
(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[ 3.0242841  14.23178    -3.1756759  ... -5.1562324  -0.95060766
  -6.757147  ]
 [ 4.4449253  -4.9428596  -0.16980551 ...  1.461592    9.603015
  -1.8790146 ]
 [ 5.059423   -4.

(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[-2.1272979  -0.79859513  3.166677   ... -3.4195323  -2.2161958
  -4.6418695 ]
 [-2.809392   -7.742143   -7.7861385  ... 12.5529585  -0.7759826
  20.317879  ]
 [-0.02603136 -1.4376576   7.4376683  ... -5.851143    1.2487596
  -4.5535364 ]
 ...
 [ 3.874576    2.418428   -5.003866   ... -4.140113   -2.5283182
  -5.046949  ]
 [-0.88608646 -4.365912   -2.7506764  ...  1.8676345   2.8000844
   0.4189559 ]
 [-1.3290924   1.0326345   4.419569   ... -6.016067   -2.3358114
  -5.3586054 ]]
<NDArray 256x10 @gpu(0)>
(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[ 2.6159172   0.84272456 -3.0681725  ... -3.0749648  -0.47109357
  -2.8017488 ]
 [-1.3438978  -4.374008   -3.498821   ... 15.633204    2.570203
   2.2902129 ]
 [ 3.5675135  -2.

...

(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[ 4.238064   -6.060924    7.2640553  ... -9.221307   -5.6084523
  -5.713896  ]
 [13.983005   -5.5847297   1.8580905  ... -4.084855   -1.9444343
  -4.029869  ]
 [ 7.1783056  -4.3901663  -5.1438284  ...  4.335662   -0.6918427
  -5.3712296 ]
 ...
 [ 0.68891716 -1.444101    5.5672126  ... -8.815612   -3.083313
  -2.4383764 ]
 [ 4.6707063  -7.3001113  -2.818031   ...  0.83525616 24.920702
   2.454986  ]
 [14.401483   -3.7361426   2.8471167  ... -6.6414547  -2.909244
  -6.8647013 ]]
<NDArray 256x10 @gpu(0)>
(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[-0.08324012 -4.274709   -4.2579846  ... 14.16938     0.9313842
   9.855507  ]
 [-1.3000858  -0.684938    8.222164   ... -6.785604   -2.0064077
   0.40001863]
 [-2.0659919  -1.848

(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[-0.2529719  -9.158791   -1.6098089  ...  6.4801283  15.361078
   3.1224806 ]
 [ 9.656951   -5.6052876   3.9717772  ... -5.1640754  -2.6086104
  -4.78018   ]
 [15.443851   -3.9029143   4.489625   ... -7.287975   -3.6814454
  -4.722691  ]
 ...
 [ 1.8416646  -7.1082726  11.861257   ... -8.24963    -3.8315938
  -2.2287264 ]
 [ 5.2022433  -9.957574    0.39302012 ...  0.7975342  16.410723
  -1.0415416 ]
 [-5.486771   -9.155162   -6.6167564  ... 16.410316    3.3871167
   7.3018312 ]]
<NDArray 256x10 @gpu(0)>
(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[-3.6396892 -4.871843  -4.3297305 ... 13.2167425  3.8838818  3.2788982]
 [-2.8152196 -4.9824214 -4.7679086 ... 15.547579  -1.9916899 14.0688715]
 [11.847037  -1.7509422  2.496290

(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[  6.2860355   -7.1680613   16.044868   ...  -9.029143    -3.016286
   -5.2107425 ]
 [  0.72601396   0.6002742    5.1013093  ... -10.145146     0.39375377
   -8.183809  ]
 [  6.8485126   -2.295837    -3.2159562  ...  -5.4510407   -2.6728039
   -5.577794  ]
 ...
 [ -3.609919    -6.036139    -4.171868   ...   8.092569     1.3691102
   21.188332  ]
 [  1.8182791   -3.31391      1.5644068  ...  -0.5059565    0.7275406
   -1.7490921 ]
 [ -0.879988    -3.0930977    3.4733605  ...  -5.2381644   -3.1742737
   -5.1787086 ]]
<NDArray 256x10 @gpu(0)>
(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[ 1.2898217  -4.77185    12.908571   ... -8.220616   -3.671663
  -3.63876   ]
 [-4.5900226  -5.86276    -3.611324   ...  2.5266669  -6.86943

(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[ 1.9792243  -3.4766085  -1.3351372  ...  2.5516407  -2.7988222
   4.3537655 ]
 [10.113755   -1.5711137  -2.3059108  ... -4.130568    0.76913416
  -5.640057  ]
 [-2.0670972  -2.9675353  -4.8793406  ... 13.996872   -1.9749272
   5.1205883 ]
 ...
 [-1.2855538  -6.13051    -3.3732457  ... 15.635129    1.3627193
   8.221455  ]
 [ 0.74344665 -4.071708   -0.7959085  ... -1.7058023   0.9094794
  -1.5004545 ]
 [ 5.816608   -8.41591    -1.4752821  ...  2.0535603  -1.33866
   8.436571  ]]
<NDArray 256x10 @gpu(0)>
(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[ 0.72777873 17.03004    -4.051508   ... -5.527609   -0.13306472
  -9.6494665 ]
 [-4.740648   -7.4241915  -4.8337426  ... 10.559986   -3.7571342
  18.248587  ]
 [-1.025538   -5.

(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[-3.3314102  -5.148885   -4.205671   ... 16.704428    0.15607259
   4.701246  ]
 [ 4.115427   -0.7218309  -3.2885513  ... -4.356282   -1.6523852
  -3.2447846 ]
 [-0.05591686 -5.5600657  -6.929892   ...  2.7551193  -0.8339329
  15.164643  ]
 ...
 [ 0.7551209  -4.0730453  12.232556   ... -8.505844   -3.7761202
  -1.8540996 ]
 [ 5.522639   -3.9880133   4.568795   ... -5.4324155  -1.3642967
  -6.3193192 ]
 [-3.3080993  -7.6920733  -4.5120873  ... 14.566962    4.5168514
  17.007114  ]]
<NDArray 256x10 @gpu(0)>
(256, 50, 5, 5)
X shape: (256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[  8.945625     0.87880075  -1.1973884  ...  -4.665031    -1.6966851
   -4.505779  ]
 [  1.2512063   -1.9454345    1.8132544  ...  -6.113807    -2.2302883
   -5.0853715 ]
 [  